BAIXAR relatórios:

REGISTRO MENSAL DE ATENDIMENTOS DOS CRAS (RMA) – FORMULÁRIO DO CRAS

Variáveis do Bloco II – Atendimentos individualizados realizados no CRAS

C1 = Quantidade total de atendimentos particularizados realizados no CRAS no mês de referência


Código para baixar os endereços dos arquivos na página

In [1]:
import os

def baixar_arquivo(url, local_filename=None):
    """Baixa e salva o arquivo de uma url."""
    # Se o nome do arquivo não foi passado como parâmetro, extrair da url.
    if local_filename is None:
        local_filename = os.path.join("./data", url.split('/')[-1])

    #verifica se arquivo já foi baixado
    if os.path.exists(local_filename):
        print(f'Arquivo {local_filename} já baixado!')
        return None
    else:
        print('Solicitando arquivo do servidor.')
        response = requests.get(url, stream=True)
        response.raise_for_status()

        with open(local_filename, 'wb') as f:
            print(f"Baixando arquivo da url: {url}")
            for chunk in response.iter_content(chunk_size=8192): 
                f.write(chunk)
            return "Arquivo baixado!"



In [2]:
def read_page(url):
    print(f'Fazendo requisição GET para o link: {url}')
    response = requests.get(url)
    response.raise_for_status() #Levanta uma exceção caso haja erro na requisição.
    
    if response.status_code == 200:
        content = response.text
        return content

Informar a página

In [3]:
import requests
from bs4 import BeautifulSoup
import time

TIME_TO_SLEEP = 120

url = 'https://aplicacoes.mds.gov.br/snas/vigilancia/index2.php'

content = read_page(url)
soup = BeautifulSoup(content, 'html.parser')

Fazendo requisição GET para o link: https://aplicacoes.mds.gov.br/snas/vigilancia/index2.php


Informar a marcação em que os endereços dos arquivos estão no HTML

In [4]:
 section = soup.find('body')
 links=section.find_all('a')
 

Todos os links da página

In [5]:
len(links)

171

In [6]:
links[10]

<a href="https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/8%20-%20POSTO%20DE%20CADASTRAMENTO.rar">POSTO DE CADASTRAMENTO</a>

Identificar apenas os links que possuem o termo CRAS

In [7]:
CRAS=[]

for item in links:
    link = item['href']
    term_link = link[75:83]
    if term_link == 'RMA_CRAS':
        CRAS.append(link)
        


In [8]:
CRAS

['https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_Criterios_2023_divulga%C3%A7%C3%A3o_030423.zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_Criterios_2022_divulga%C3%A7%C3%A3o_27_03_23.xlsx',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_Criterios_2021_divulga%C3%A7%C3%A3o.zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_Criterios_2020_divulga%C3%A7%C3%A3o(2).zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_NovoCriterios_2019_divulga%C3%A7%C3%A3o.zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_NovoCriterios_2018_divulga%C3%A7%C3%A3o.zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_NovoCriterios_2017_divulga%C3%A7%C3%A3o.zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/f

Pegar apenas o link de 2022 - segundo arquivo

In [9]:
for item in CRAS[1:2]:
    if baixar_arquivo(item):
        print('Esperando 120s para solicitar próximo download...')
        time.sleep(TIME_TO_SLEEP)
    else:
        print("Link para download não encontrado.")         

Arquivo ./data\RMA_CRAS_Criterios_2022_divulga%C3%A7%C3%A3o_27_03_23.xlsx já baixado!
Link para download não encontrado.


Descompactar o arquivo - ZIP na pasta data

In [10]:
#import os
#import zipfile

##caminho = os.getcwd()
#caminho = 'data'
#arquivos = os.listdir(caminho)

In [11]:
#for arquivo in arquivos:
#    if arquivo.endswith('.zip'):
#        caminho_completo = os.path.join(caminho, arquivo)
#
#        with zipfile.ZipFile(caminho_completo, 'r') as zip_ref:
#            zip_ref.extractall(caminho)

In [12]:
import pandas as pd 


In [13]:
#arquivos=[]
#for file in os.listdir(caminho):
#    if file.endswith(".xlsx"):
#        print(f'Carregando arquivo {file} ...')
#        arquivos.append(pd.read_excel(os.path.join(caminho,file), sheet_name="Base tratada"))

Buscar apenas os arquivos terminados em XLSX na pasta data

In [14]:
import glob
caminho = 'data'
os.chdir(caminho)
arquivos2 = glob.glob('*.xlsx')


Juntar arquivos em um dataframe

In [15]:
arquivos2

['Censo_2010_pop.xlsx',
 'RMA_CRAS_Criterios_2022_divulga%C3%A7%C3%A3o_27_03_23.xlsx']

Abrir apenas uma aba especifica do XLSX

In [16]:
tabelas=[]

for x in arquivos2:
    dados = pd.ExcelFile(x)
    print(f'Carregando arquivo {dados} ...')
    tabelas.append(pd.read_excel(dados,sheet_name="Base tratada"))

print('fim')
   
    

Carregando arquivo <pandas.io.excel._base.ExcelFile object at 0x000001CFF3D59250> ...


ValueError: Worksheet named 'Base tratada' not found

In [ ]:
len(tabelas)

In [ ]:
tabela_final=pd.concat(tabelas)

In [ ]:
tabela_final.head(2)

In [ ]:
tabela_final.columns

In [ ]:
dados_municipio=tabela_final[['municipio', 'uf', 'IBGE','IBGE7','Nome_UF', 'Região', 'Município', 'Pop_total_2010', 'Pop_Urbana2010', 'Pop_Rural2010', 'areaKm2', 'Porte_pop2010']]

In [ ]:
dados_municipio=dados_municipio.drop_duplicates(subset=['municipio','uf','IBGE','IBGE7'], keep='last')

In [ ]:
dados_municipio

Porte do municipio

In [ ]:
url='https://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista_Munic%C3%ADpios_com_IBGE_Brasil_Versao_CSV.csv'
porte=pd.read_csv(url, sep=";", encoding='Latin-1')

In [ ]:
porte=porte[['IBGE','IBGE7','UF','Município','Região','População 2010','Porte']]

In [ ]:
porte

In [ ]:
Dados_municipio_porte = pd.merge(porte, dados_municipio, how = 'left', 
                                 right_on=['IBGE7',],
                                 left_on=['IBGE7',])

In [ ]:
Dados_municipio_porte.sort_values(by="IBGE_y")

Municipios não encontrados no RMA 2022

In [ ]:
MuncipiosFalta=Dados_municipio_porte[Dados_municipio_porte['municipio'].isna()]

In [ ]:
MuncipiosFalta

In [ ]:
MuncipiosFalta.info()

In [ ]:
RMA_mensal=tabela_final.loc[:,['ano', 'mes', 'municipio', 'uf', 'c1', 'c1_original', 'IBGE','IBGE7']]

In [ ]:
RMA_mensal.head(2)

In [ ]:
RMA_mensal.info()

In [ ]:
RMA_m_2022=RMA_mensal[(RMA_mensal['ano']==2022)]

In [ ]:
RMA_m_2022.info()

In [ ]:
RMA_2022_0=RMA_m_2022.groupby(['ano','mes','municipio','uf','IBGE','IBGE7']).sum().reset_index()

In [ ]:
RMA_2022_0.info()

In [ ]:
RMA_2022_soma=RMA_2022_0.groupby(['ano','municipio','uf','IBGE','IBGE7']).sum().reset_index()
RMA_2022_soma=RMA_2022_soma.rename(columns={'c1':'soma_c1', 'c1_original':'soma_c1_original', }).drop(columns={'mes'})
RMA_2022_media=RMA_2022_0.groupby(['ano','municipio','uf','IBGE','IBGE7']).mean().round(1).reset_index()
RMA_2022_media=RMA_2022_media.rename(columns={'c1':'media_c1', 'c1_original':'media_c1_original', }).drop(columns={'mes'})

In [ ]:
RMA_2022_media.tail()

In [ ]:
RMA_2022_soma.tail()

In [ ]:
RMA_2022=pd.merge(RMA_2022_soma,RMA_2022_media, how = 'inner', 
                  right_on=['ano', 'municipio','IBGE7','IBGE','uf'],
                  left_on=['ano', 'municipio','IBGE7','IBGE','uf'])
RMA_2022.tail()

In [ ]:
RMA_2022[(RMA_2022['IBGE']==530010)]

In [ ]:
RMA_m_2022_CRAS=tabela_final.loc[:,['ano', 'mes', 'municipio','NU_IDENTIFICADOR', 'uf', 'IBGE','IBGE7']]
RMA_m_2022_CRAS=RMA_m_2022_CRAS[(RMA_m_2022_CRAS['ano']==2022)]
RMA_m_2022_CRAS

In [ ]:
RMA_m_2022_CRAS=RMA_m_2022_CRAS.groupby(['ano','mes','municipio','uf','IBGE','IBGE7'])['NU_IDENTIFICADOR'].agg(['count']).reset_index()
RMA_m_2022_CRAS

In [ ]:
RMA_2022_CRAS=RMA_m_2022_CRAS.groupby(['ano','municipio','uf','IBGE','IBGE7'])['count'].agg(['max']).reset_index()
RMA_2022_CRAS=RMA_2022_CRAS.rename(columns={'max':'total_max_CRAS'})

In [ ]:
RMA_2022_CRAS

Consistencia numero de CRAS

In [ ]:
RMA_m_2022_CRAS[(RMA_m_2022_CRAS['IBGE']==530010)]

In [ ]:
RMA_2022_CRAS[(RMA_2022_CRAS['IBGE']==530010)]


In [ ]:
RMA_mensal=tabela_final[['ano', 'mes', 'municipio', 'uf', 'c1', 'c1_original', 'IBGE','IBGE7']]

In [ ]:
RMA_m_12_2022=RMA_mensal[((RMA_mensal['mes']==12) & (RMA_mensal['ano']==2022))]
RMA_m_12_2022

In [ ]:
RMA_m_12_2022=RMA_m_12_2022.groupby(['municipio','uf','IBGE','IBGE7']).sum().reset_index()
RMA_m_12_2022=RMA_m_12_2022.rename(columns={'c1':'c1_DEZ_2022','c1_original':'c1_original_DEZ_2022'})
RMA_m_12_2022=RMA_m_12_2022.drop(columns={'ano','mes'})
RMA_m_12_2022

In [ ]:
RMAcompleto0 = pd.merge(RMA_2022, RMA_2022_CRAS, how = 'left')

In [ ]:
RMAcompleto0 

In [ ]:
RMAcompleto= pd.merge(RMAcompleto0, RMA_m_12_2022,how='left')
RMAcompleto

In [ ]:
RMAcompleto[(RMAcompleto['IBGE']==530010)]

In [ ]:
RMAcompleto.sort_values(by=['soma_c1'], ascending=False)

In [ ]:
RMAcompleto_Nulos=RMAcompleto[(RMAcompleto['soma_c1_original'] == 0 )].sort_values(by=['soma_c1_original'], ascending=False)

In [ ]:
RMAcompleto_Nulos

In [ ]:
RMAcompleto_nao_Nulos=RMAcompleto[(RMAcompleto['soma_c1_original'] != 0 )].sort_values(by=['soma_c1_original'], ascending=False)          

In [ ]:
RMAcompleto_nao_Nulos

In [ ]:
RMA_2022_porte=pd.merge(RMAcompleto_nao_Nulos,Dados_municipio_porte, 
                        how = 'inner')

In [ ]:
RMA_2022_porte.columns

In [ ]:
RMA_TOTAL_2022=RMA_2022_porte[['ano', 'municipio','Município_x', 'uf', 'IBGE', 'IBGE7', 'soma_c1',
       'soma_c1_original', 'media_c1', 'media_c1_original', 'total_max_CRAS',
       'c1_DEZ_2022', 'c1_original_DEZ_2022', 'Porte']]

In [ ]:
RMA_TOTAL_2022=RMA_TOTAL_2022.rename(columns={'IBGE':'ibge_6',	'IBGE7':'ibge_7','Município_x':'municipio_aj','uf':'uf',	
                                              'total_max_CRAS':'total_max_cras','Porte':'porte',
                                             'c1_DEZ_2022': 'c1_dez_2022', 'c1_original_DEZ_2022':'c1_original_dez_2022'})

In [ ]:
RMA_TOTAL_2022

10 maiores RMA

In [ ]:
RMA_TOTAL_2022.sort_values(by=['soma_c1'], ascending=False)[0:10]

10 menores RMA

In [ ]:
RMA_TOTAL_2022.sort_values(by=['soma_c1'], ascending=True)[0:10]

In [ ]:
RMA_TOTAL_2022.describe()

In [ ]:
#RMA_TOTAL_2022.to_csv('RMA_TOTAL_2022.csv',encoding = 'UTF-8')

In [ ]:
url_cad_2022 = 'https://aplicacoes.mds.gov.br/sagi/servicos/misocial?fq=anomes_s:2022*&wt=csv&omitHeader=true&fq=cadunico_tot_fam_i:{0%20TO%20*]&q=*&fl=ibge:codigo_ibge,anomes:anomes_s,cadunico_tot_fam:cadunico_tot_fam_i,cadunico_tot_pes:cadunico_tot_pes_i,cadunico_tot_fam_rpc_ate_meio_sm:cadunico_tot_fam_rpc_ate_meio_sm_i,cadunico_tot_pes_rpc_ate_meio_sm:cadunico_tot_pes_rpc_ate_meio_sm_i,cadunico_tot_fam_pob:cadunico_tot_fam_pob_i,cadunico_tot_pes_pob:cadunico_tot_pes_pob_i,cadunico_tot_fam_ext_pob:cadunico_tot_fam_ext_pob_i,cadunico_tot_pes_ext_pob:cadunico_tot_pes_ext_pob_i,cadunico_tot_fam_pob_e_ext_pob:cadunico_tot_fam_pob_e_ext_pob_i,cadunico_tot_pes_pob_e_ext_pob:cadunico_tot_pes_pob_e_ext_pob_i&rows=100000000&sort=anomes_s%20desc,%20codigo_ibge%20asc'

In [ ]:
cad_2022=pd.read_csv(url_cad_2022, sep=",", encoding='Latin-1')

In [ ]:
cad_2022

In [ ]:
cad_2022.columns

In [ ]:
cad_2022['total_bx_renda'] = cad_2022['cadunico_tot_pes_rpc_ate_meio_sm']-cad_2022['cadunico_tot_pes_pob_e_ext_pob']


In [ ]:
cad_2022.columns

In [ ]:
cad_2022=cad_2022.rename(columns={'ibge':'ibge_6', 
                             'cadunico_tot_pes':'tot_pes','cadunico_tot_pes_rpc_ate_meio_sm':'tot_pes_rpc_ate_meio_sm', 
                             'cadunico_tot_pes_pob':'tot_pes_pob','cadunico_tot_pes_ext_pob':'tot_pes_ext_pob', 
                             'cadunico_tot_pes_pob_e_ext_pob':'tot_pes_pob_e_ext_pob','total_bx_renda':'tot_bx_renda'})

In [ ]:
cad_2022

In [ ]:
cad_2022_Ano=cad_2022[['ibge_6', 'anomes', 'tot_pes', 'tot_pes_rpc_ate_meio_sm', 'tot_pes_pob','tot_pes_ext_pob', 'tot_pes_pob_e_ext_pob', 'tot_bx_renda']]

In [ ]:
cad_2022_Ano

In [ ]:
cad_2022_Soma=cad_2022_Ano.groupby(by=['ibge_6']).sum()

In [ ]:
cad_2022_Soma.columns

In [ ]:
cad_2022_Soma=cad_2022_Soma.rename(columns={'tot_pes':'Soma_tot_pes', 
                                    'tot_pes_rpc_ate_meio_sm':'Soma_tot_pes_rpc_ate_meio_sm', 
                                    'tot_pes_pob':'Soma_tot_pes_pob',
                                    'tot_pes_ext_pob':'Soma_pes_ext_pob', 
                                    'tot_pes_pob_e_ext_pob':'Soma_tot_pes_pob_e_ext_pob', 
                                    'tot_bx_renda':'Soma_tot_bx_renda'})

In [ ]:
cad_2022_Soma_=cad_2022_Soma.drop(columns={'anomes'}).reset_index()

In [ ]:
cad_2022_Soma_

In [ ]:
cad_2022_Media=cad_2022_Ano.groupby(by=['ibge_6']).mean().round(2)

In [ ]:
cad_2022_Media.columns

In [ ]:
cad_2022_Media=cad_2022_Media.rename(columns={'tot_pes':'Media_tot_pes', 
                                    'tot_pes_rpc_ate_meio_sm':'Media_tot_pes_rpc_ate_meio_sm', 
                                    'tot_pes_pob':'Media_tot_pes_pob',
                                    'tot_pes_ext_pob':'Media_pes_ext_pob', 
                                    'tot_pes_pob_e_ext_pob':'Media_tot_pes_pob_e_ext_pob', 
                                    'tot_bx_renda':'Media_tot_bx_renda'})

In [ ]:
cad_2022_Media=cad_2022_Media.drop(columns={'anomes'}).reset_index()

In [ ]:
cad_2022_Media

In [ ]:
cad_2022_Dez=cad_2022_Ano[(cad_2022_Ano['anomes']== 202212)]

In [ ]:
cad_2022_Dez.columns

In [ ]:
cad_2022_Dez=cad_2022_Dez.rename(columns={'tot_pes':'Dez_tot_pes', 
                                    'tot_pes_rpc_ate_meio_sm':'Dez_tot_pes_rpc_ate_meio_sm', 
                                    'tot_pes_pob':'Dez_tot_pes_pob',
                                    'tot_pes_ext_pob':'Dez_pes_ext_pob', 
                                    'tot_pes_pob_e_ext_pob':'Dez_tot_pes_pob_e_ext_pob', 
                                    'tot_bx_renda':'Dez_tot_bx_renda'})

In [ ]:
cad_2022_Dez=cad_2022_Dez.drop(columns={'anomes'})

In [ ]:
cad_2022_Dez

In [ ]:
cad2022_0=pd.merge(cad_2022_Dez,cad_2022_Media, how = 'inner', right_on='ibge_6',left_on='ibge_6').reset_index()

In [ ]:
cad2022=pd.merge(cad2022_0,cad_2022_Soma_, how = 'inner', right_on='ibge_6',left_on='ibge_6')

In [ ]:
cad2022

<h2>Tabela RMA_comparativo com dados do CADUnico, dados de RMA e porte</h2>

In [ ]:
RMA_2022_comparativo0=pd.merge(cad2022,RMA_TOTAL_2022, how = 'left', right_on='ibge_6',left_on='ibge_6')

In [ ]:
RMA_2022_comparativo=pd.merge(RMA_2022_comparativo0, porte, how = 'left', right_on='IBGE',left_on='ibge_6')

In [ ]:
RMA_2022_comparativo

In [ ]:
#RMA_2022_comparativo.to_csv('RMA_2022_comparativo.csv',encoding = 'UTF-8')

In [ ]:
RMA_2022_comparativo.columns

In [ ]:
RMA_2022_comparativo_media=RMA_2022_comparativo[['ibge_6','ibge_7','municipio_aj', 'total_max_cras','soma_c1','media_c1', 'Dez_tot_pes_pob_e_ext_pob','Porte']]
#RMA_2022_comparativo_media=RMA_2022_comparativo_media[(RMA_2022_comparativo_media['media_c1']!=0)]
RMA_2022_comparativo_media.describe()

<h3>Inserção da coluna:</h3>
</br> 
<p><strong>RMA médio dividido por número de pessoas com bolsa família em pobreza e extrema pobreza de dez/2022 multiplicado por 100 </strong></p>
RMA-media_c1 /  Dez_tot_pes_pob_e_ext_pob * 100	 </h3>

In [ ]:
RMA_2022_comparativo_media.sort_values(by='Dez_tot_pes_pob_e_ext_pob')

In [ ]:
RMA_2022_comparativo_media[RMA_2022_comparativo_media['media_c1'].isna()]

In [ ]:
RMA_2022_comparativo_media = RMA_2022_comparativo_media.fillna(0)

In [ ]:
RMA_2022_comparativo_media[RMA_2022_comparativo_media['media_c1'].isna()]

In [ ]:
RMA_2022_comparativo_media

In [ ]:
RMA_2022_comparativo_media.describe()

In [ ]:
RMA_2022_comparativo_media[RMA_2022_comparativo_media['media_c1'].isna()]

In [ ]:
RMA_2022_comparativo_media['RMA_media_div_pob_e_ext_pob']=RMA_2022_comparativo_media['media_c1']/RMA_2022_comparativo_media['Dez_tot_pes_pob_e_ext_pob']*100

In [ ]:
RMA_2022_comparativo_media.sort_values(by='soma_c1')

In [ ]:
RMA_2022_comparativo_media.describe()

<h2>Definição das classes</h2>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
dados=RMA_2022_comparativo_media

In [ ]:
dados.info()

In [ ]:
dados[['RMA_media_div_pob_e_ext_pob']].boxplot()


In [ ]:
indice_max = dados['RMA_media_div_pob_e_ext_pob'].idxmax()
linha_max = dados['RMA_media_div_pob_e_ext_pob'].loc[indice_max]
linha_max

In [ ]:
maiores_valores = dados['RMA_media_div_pob_e_ext_pob'].nlargest(10)
menores_valores = dados['RMA_media_div_pob_e_ext_pob'].nsmallest(10)

In [ ]:
maiores_valores

In [ ]:
menores_valores

In [ ]:
fig = plt.figure(figsize=(12,7))

ax1 = plt.subplot(2, 2, 1)
ax1.set_title('Matplotlib')
ax1.hist(x=dados['RMA_media_div_pob_e_ext_pob']);

ax2 = plt.subplot(2, 2, 2, sharey=ax1)
ax2.set_title('Pandas')
dados.plot.hist(column='RMA_media_div_pob_e_ext_pob', bins=10, ax=ax2);

ax3 = plt.subplot(2, 1, 2)
plt.title('Seaborn')

sns.histplot(data=dados, x='RMA_media_div_pob_e_ext_pob', 
             multiple='stack', bins=100, ax=ax3);
             

In [ ]:
dados['indice'] = pd.cut(dados['RMA_media_div_pob_e_ext_pob'], bins = [-1, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 850])

In [ ]:
indice_11c = dados['indice'].value_counts()
indice_11c = pd.DataFrame(indice_11c).sort_values(by='indice')
indice_11c 

In [ ]:
df_indice_RMA = indice_11c.reset_index(drop=False).copy()
df_indice_RMA

In [ ]:
df_indice_RMA['Classe'] = [0,1,2,3,4,5,6,7,8,9,10,11]
df_indice_RMA

In [ ]:
fig10c, ax = plt.subplots()

X = df_indice_RMA['Classe']
Y = df_indice_RMA['count']
bars = ax.bar(X, Y)

plt.title('Relação entre o RMA -media_c1- e pessoas_pobreza_extrema_pobreza')

ax.set_xlabel('Relação entre o RMA -media_c1 - e pessoas_pobreza_extrema_pobreza', fontsize=8)
ax.set_ylabel('Número de Municípios', fontsize=8)

ax.set_xticklabels(['0','0','5-10','15-20','25-30','35-40','45-50'],
                   rotation=90, fontsize=6)

# Adicione etiquetas em todas as barras
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),  # Deslocamento vertical das etiquetas
                textcoords="offset points",
                ha='center', va='bottom', fontsize=6
               )
plt.xticks(rotation=50, fontsize=8)

plt.show()


In [ ]:
dados.sort_values(by='Dez_tot_pes_pob_e_ext_pob')

In [ ]:
dados[(dados['ibge_7']==5300108)]

In [ ]:
dados[(dados['ibge_7']==1100015)]

In [ ]:
dadosPorte=pd.merge(dados,df_indice_RMA, how='inner')
dadosPorte=dadosPorte.drop(columns={'count'}).rename(columns={'indice':'faixa'})
dadosPorte.sort_values(by='RMA_media_div_pob_e_ext_pob')[-360:-355]

<h1>Filtro colunas importantes para estudo do indice</h1>

In [ ]:
RMA_classe=dadosPorte[['ibge_6','RMA_media_div_pob_e_ext_pob','Classe']]

In [ ]:
RMA_classe.describe()

In [ ]:
RMA_classe.sort_values(by='RMA_media_div_pob_e_ext_pob')

In [ ]:
dadosPorte.to_csv('dadosPorte_classe.csv',encoding = 'UTF-8')

In [ ]:
grafico = dadosPorte['Classe'].value_counts()
grafico = pd.DataFrame(grafico).sort_values(by=['Classe'])
df_grafico = grafico.reset_index(drop=False).copy()
print(df_grafico)
m=sns.barplot(data = df_grafico, y = 'count', x = 'Classe')
m.set_xticklabels(m.get_xticklabels(), rotation=30)

<h2>Gráficos por porte</h2>

In [ ]:
dadosPorteMetropole=dadosPorte[(dadosPorte['Porte']=='Metrópole')]
dadosPorteMetropole

In [ ]:
Metropole = dadosPorteMetropole['faixa'].value_counts()
Metropole = pd.DataFrame(Metropole).sort_values(by=['faixa'])
df_indice_Metropole = Metropole.reset_index(drop=False).copy()
m=sns.barplot(data = df_indice_Metropole, y = 'count', x = 'faixa')
m.set_xticklabels(m.get_xticklabels(), rotation=30)

In [ ]:
dadosPorteGrande=dadosPorte[(dadosPorte['Porte']=='Grande')]
dadosPorteGrande.head()

In [ ]:
Grande = dadosPorteGrande['faixa'].value_counts()
Grande = pd.DataFrame(Grande).sort_values(by=['faixa'])
df_indice_Grande = Grande.reset_index(drop=False).copy()
g=sns.barplot(data = df_indice_Grande, y = 'count', x ='faixa')
g.set_xticklabels(g.get_xticklabels(), rotation=30)

In [ ]:
dadosPorteMedio=dadosPorte[(dadosPorte['Porte']=='Médio')]
dadosPorteMedio.head()

In [ ]:
Medio = dadosPorteMedio['faixa'].value_counts()
Medio = pd.DataFrame(Medio).sort_values(by=['faixa'])
df_indice_Medio = Medio.reset_index(drop=False).copy()
print(df_indice_Medio)
med=sns.barplot(data = df_indice_Medio, y = 'count', x ='faixa')
med.set_xticklabels(med.get_xticklabels(), rotation=30)

In [ ]:
dadosPortePequenoI=dadosPorte[(dadosPorte['Porte']=='Pequeno I')]
dadosPortePequenoI.head()

In [ ]:
PequenoI= dadosPortePequenoI['faixa'].value_counts()
PequenoI = pd.DataFrame(PequenoI).sort_values(by=['faixa'])
df_indice_PequenoI = PequenoI.reset_index(drop=False).copy()
PEQ1=sns.barplot(data = df_indice_PequenoI, y = 'count', x ='faixa')
PEQ1.set_xticklabels(PEQ1.get_xticklabels(), rotation=30)

In [ ]:
dadosPortePequenoII=dadosPorte[(dadosPorte['Porte']=='Pequeno II')]
dadosPortePequenoII.head()

In [ ]:
PequenoII= dadosPortePequenoII['faixa'].value_counts()
PequenoII = pd.DataFrame(PequenoII).sort_values(by=['faixa'])
df_indice_PequenoII = PequenoII.reset_index(drop=False).copy()
PEQ2=sns.barplot(data = df_indice_PequenoII, y = 'count', x ='faixa')
PEQ2.set_xticklabels(PEQ2.get_xticklabels(), rotation=30)

In [ ]:
dadosPorte['Porte'].value_counts()

In [ ]:
fig = plt.figure(figsize=(12,7))

ax1 = plt.subplot(3, 3, 1)
sns.barplot(data = df_indice_Metropole, y = 'count', x ='faixa',ax=ax1)
plt.xticks(rotation=30)

plt.title('Metropole')

ax2 = plt.subplot(3, 3, 2,sharey=ax1)
sns.barplot(data = df_indice_Grande, y = 'count', x ='faixa',ax=ax2)
plt.title('Grande')
plt.xticks(rotation=30)

ax3 = plt.subplot(3, 3, 3,sharey=ax2)
sns.barplot(data = df_indice_Medio, y = 'count', x ='faixa',ax=ax3)
plt.title('Medio')
plt.xticks(rotation=30)


fig = plt.figure(figsize=(12,7))

ax1 = plt.subplot(2, 2, 1)
sns.barplot(data = df_indice_PequenoII, y = 'count', x ='faixa',ax=ax1)
plt.xticks(rotation=30)
plt.title('PequenoII')

ax2 = plt.subplot(2, 2, 2,sharey=ax1)
sns.barplot(data = df_indice_PequenoI, y = 'count', x ='faixa',ax=ax2)
plt.title('PequenoI')
plt.xticks(rotation=30)